In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from pathlib import Path

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
tf.version

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
path  = Path('../input/small-jpegs-fgvc')
path

In [ ]:
train_df = pd.read_csv(path/'train_cultivar_mapping.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.is_exist.unique()

In [ ]:
IMG_SIZE = 256
BATCH_SIZE = 32

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=path/'train',
    x_col='image',
    y_col='cultivar',
    class_mode='sparse',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    subset='training',
    shuffle=True
)

valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=path/'train',
    x_col='image',
    y_col='cultivar',
    class_mode='sparse',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation',
    shuffle=True
)

test_datagen = ImageDataGenerator()

In [ ]:
base_model = keras.applications.EfficientNetB4(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
from keras.layers import Dense, GlobalAveragePooling2D, Input

inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
output = base_model(inputs)
output = GlobalAveragePooling2D()(output)
output = Dense(100, activation='softmax')(output)

model = keras.Model(inputs, output)

model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:

model.fit(train_generator, validation_data=valid_generator, epochs=10)

In [ ]:
import os

test_filenames = os.listdir(path/'test')
test_df = pd.DataFrame({'filename' : test_filenames})

test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = path/'test',
    x_col = 'filename',
    y_col = None,
    class_mode = None,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = False
)

In [ ]:
classes = list(train_df.cultivar.unique())
len(classes)

In [ ]:
import numpy as np

preds = model.predict(test_generator)
pred_idx = np.argmax(preds, axis=1)
predictions = [classes[i] for i in pred_idx]

In [ ]:
test_df['cultivar'] = predictions

In [ ]:
test_df.shape

In [ ]:
test_df.filename = test_df.filename.map(lambda x: x.split('.jpeg')[0] + '.png')

In [ ]:
test_df.head()

In [ ]:
test_df.to_csv('submission.csv', index=False)